In [22]:
import csv


In [26]:
def fixDataset(csvName):
    ids_1 = {"count":0}
    ids_2 = {"count":0}
    ids_3 = {"count":0}
    ids_4 = {"count":0}

    with open(csvName, 'r') as f:
        reader = csv.reader(f)
        text_file = open("newCsvDataset.txt", "w")
        for row in reader:
            if row[0] == "duration":
                for item in row:
                    text_file.write("{},".format(item))
            else:
                for row_i in range(len(row)):
                    text = ""
                    if row_i == 1:
                        if row[row_i] in ids_1:
                            text = str(ids_1[row[row_i]])
                        else:
                            ids_1[row[row_i]] = ids_1["count"] 
                            ids_1["count"] += 1
                            text = str(ids_1[row[row_i]])
                            
                    elif row_i == 2:
                        if row[row_i] in ids_2:
                            text = str(ids_2[row[row_i]])
                        else:
                            ids_2[row[row_i]] = ids_2["count"] 
                            ids_2["count"] += 1
                            text = str(ids_2[row[row_i]])
                    elif row_i == 3:
                        if row[row_i] in ids_3:
                            text = str(ids_3[row[row_i]])
                        else:
                            ids_3[row[row_i]] = ids_3["count"] 
                            ids_3["count"] += 1
                            text = str(ids_3[row[row_i]])
                    elif row_i == 41:
                        if row[row_i] in ids_4:
                            text = str(ids_4[row[row_i]])
                        else:
                            ids_4[row[row_i]] = ids_4["count"] 
                            ids_4["count"] += 1
                            text = str(ids_4[row[row_i]])
                    else:
                        text = row[row_i]
                        
                    text_file.write("{},".format(text))
            
    return [ids_1,ids_2,ids_3,ids_4]

In [28]:
fixDataset("kddtest.csv")

[{'count': 3, 'tcp': 0, 'icmp': 1, 'udp': 2},
 {'count': 64,
  'private': 0,
  'ftp_data': 1,
  'eco_i': 2,
  'telnet': 3,
  'http': 4,
  'smtp': 5,
  'ftp': 6,
  'ldap': 7,
  'pop_3': 8,
  'courier': 9,
  'discard': 10,
  'ecr_i': 11,
  'imap4': 12,
  'domain_u': 13,
  'mtp': 14,
  'systat': 15,
  'iso_tsap': 16,
  'other': 17,
  'csnet_ns': 18,
  'finger': 19,
  'uucp': 20,
  'whois': 21,
  'netbios_ns': 22,
  'link': 23,
  'Z39_50': 24,
  'sunrpc': 25,
  'auth': 26,
  'netbios_dgm': 27,
  'uucp_path': 28,
  'vmnet': 29,
  'domain': 30,
  'name': 31,
  'pop_2': 32,
  'http_443': 33,
  'urp_i': 34,
  'login': 35,
  'gopher': 36,
  'exec': 37,
  'time': 38,
  'remote_job': 39,
  'ssh': 40,
  'kshell': 41,
  'sql_net': 42,
  'shell': 43,
  'hostnames': 44,
  'echo': 45,
  'daytime': 46,
  'pm_dump': 47,
  'IRC': 48,
  'netstat': 49,
  'ctf': 50,
  'nntp': 51,
  'netbios_ssn': 52,
  'tim_i': 53,
  'supdup': 54,
  'bgp': 55,
  'nnsp': 56,
  'rje': 57,
  'printer': 58,
  'efs': 59,
  'X11'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
# from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
# from pandas_ml import ConfusionMatrix
from sklearn.metrics import confusion_matrix
print("***************************************************************")

traindata = pd.read_csv('kddtrain-20-mod.csv', dtype={"protocol_type":np.str, "service": np.str, "flag": np.str, "attack_type": np.str}, header=0)
testdata = pd.read_csv('kddtest-mod.csv', dtype={"protocol_type":np.str, "service": np.str, "flag": np.str, "attack_type": np.str}, header=0)

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)


traindata = np.array(trainX)
trainlabel = np.array(Y)

testdata = np.array(testT)
testlabel = np.array(C)


model = LogisticRegression()
model.fit(traindata, trainlabel)


# make predictions
expected = testlabel
predicted = model.predict(testdata)


print("***************************************************************")



# fit a Naive Bayes model to the data
model = GaussianNB()
model.fit(traindata, trainlabel)
print(model)
# make predictions
expected = testlabel
predicted = model.predict(testdata)

cm = confusion_matrix(expected, predicted)
print(expected.shape)
print(predicted.shape)
expected = np.array(expected)
predicted = np.array(predicted)
cm.print_stats()

np.savetxt('expected.txt', expected, fmt='%01d')
np.savetxt('predicted.txt',predicted , fmt='%01d')

print(cm)
print(expected.shape)
print(predicted.shape)
cm.stats()
print("***************************************************************")




***************************************************************


In [ ]:

# fit a k-nearest neighbor model to the data
model = KNeighborsClassifier()
model.fit(traindata, trainlabel)
print(model)
# make predictions
expected = testlabel
predicted = model.predict(testdata)
# summarize the fit of the model

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("recall")
print("%.3f" %recall)
print("f-score")
print("%.3f" %f1)
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)
print("***************************************************************")



model = DecisionTreeClassifier()
model.fit(traindata, trainlabel)
print(model)
# make predictions
expected = testlabel
predicted = model.predict(testdata)
# summarize the fit of the model

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("recall")
print("%.3f" %recall)
print("f-score")
print("%.3f" %f1)
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)
print("***************************************************************")






model = AdaBoostClassifier(n_estimators=100)
model.fit(traindata, trainlabel)

# make predictions
expected = testlabel
predicted = model.predict(testdata)
# summarize the fit of the model

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("recall")
print("%.3f" %recall)
print("f-score")
print("%.3f" %f1)
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)
print("***************************************************************")




model = RandomForestClassifier(n_estimators=100)
model = clf.fit(traindata, trainlabel)

# make predictions
expected = testlabel
predicted = model.predict(testdata)
# summarize the fit of the model

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("recall")
print("%.3f" %recall)
print("f-score")
print("%.3f" %f1)
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)
print("***************************************************************")
